In [3]:
import pandas as pd
import random
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical


# Load the data
df = pd.read_csv("lamp_status_data.csv")

# Process the data
df['time'] = pd.to_datetime(df['time']).dt.hour
# df['time'] = df['time'].apply(lambda x: x.hour*60 + x.minute)

le = LabelEncoder()
df['room'] = le.fit_transform(df['room'])

# Prepare data for LSTM
X = df[['room', 'time']].values
y = df['lamp_status'].values

# Normalize X
X = X / np.max(X, axis=0)

# Reshape X to be 3D [samples, timesteps, features] for LSTM
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Convert y to one-hot encoding
y = to_categorical(y)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(2, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


[[ 4 19]
 [ 3 19]
 [ 1 19]
 ...
 [ 2  7]
 [ 3  7]
 [ 0  7]]


In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=4096, validation_data=(X_test, y_test), shuffle=False)


In [ ]:
import joblib 
model.save('lamp_status_model.h5')
joblib.dump(le, 'label_encoder.pkl')
